In [97]:
def calculate_amortization_amount(principal, interest_rate, periods):
    x = (1 + interest_rate) ** periods
    return principal * (interest_rate * x) / (x - 1)

def amortization_schedule(principal, interest_rate, periods, additional = 0):
    amortization_amount = calculate_amortization_amount(principal, interest_rate, periods)
    number = 1
    balance = principal
    for period in range(periods):
        interest = balance * interest_rate
        principal = amortization_amount - interest
        balance = balance - principal - additional
        yield (period + 1), amortization_amount, interest, principal, balance if balance > 0 else 0
        


In [2]:
import pandas as pd

In [107]:
input_down = 125000
purchase_price = 640000
interest_rate = 3.875 / 100
interest_rate_monthly = interest_rate / 12
years = 30
months = years * 12
periods = months
term = 'month'
tax_rate = 0.011
tax_rate_monthly = tax_rate / 12

alternative = 3500
expected_sale = 700000
deduction_cap = 10000
deduction_monthly = deduction_cap / 12
hoa = 321
down_amount = 0 if input_down is None else input_down
down_percent = 0.20 if down_amount == 0 else down_amount / purchase_price
principal = purchase_price * (1 - down_percent)

In [113]:
df = pd.DataFrame(amortization_schedule(principal, interest_rate_monthly, periods, 1000), columns = [term, 'amortization', 'interest', 'principal', 'balance'])
df.set_index(keys = term, inplace = True)

df['down'] = down_amount
df['expected_sale'] = expected_sale

df['interest+principal'] = df.interest + df.principal
df['cumulative_interest'] = df.interest.cumsum(axis = 'index')
df['cumulative_principal'] = df.principal.cumsum(axis = 'index')

df['tax'] = tax_rate_monthly * purchase_price
df['cumulative_tax'] = df.tax.cumsum(axis = 'index')
df['deduction'] = deduction_monthly
df['cumulative_deduction'] = df.deduction.cumsum(axis = 'index')
df['alternative'] = alternative
df['cumulative_alternative'] = df.alternative.cumsum(axis = 'index')

df['hoa'] = hoa
df['cumulative_hoa'] = df.hoa.cumsum(axis = 'index')

df['expense'] = df.tax + df.hoa + df.interest - df.alternative
df['cumulative_expense'] = df.expense.cumsum(axis = 'index')

sales_commission = 5 / 100
fees = 10000
df['equity'] = df.cumulative_principal + down_amount
df['min_sale_price'] = (df.down + df.balance + df.cumulative_expense) / (1 - sales_commission) + fees - df.deduction

# calculate balance for accounting additional principal payments
# df['balance_check'] = df['balance'].shift(-1) - df.principal.shift(-1)

df = df[[
    'amortization'
    ,'interest+principal'
    ,'interest' ,'cumulative_interest'
    ,'principal', 'cumulative_principal'
    ,'hoa', 'cumulative_hoa'
    ,'tax', 'cumulative_tax'
    ,'deduction', 'cumulative_deduction'
    ,'equity'
    ,'expense', 'cumulative_expense'
    ,'alternative', 'cumulative_alternative'
    ,'balance'
    ,'expected_sale'
    ,'down'
    ,'min_sale_price'
]]

crossover_period = 0
crossover_amount = 0
payment_prior = None
for period, payment in df.iterrows():
    if period > 1 and (payment_prior.cumulative_principal <= payment_prior.cumulative_expense
        and payment.cumulative_expense <= payment.cumulative_principal):
        crossover_period = period
        crossover_amount = payment.equity
    payment_prior = payment

df['milestones'] = ''
df.loc[crossover_period, 'milestones'] = 'crossover'
print(crossover_period)
print(f"mean monthly: {df.amortization.mean():.2f}")
df.head()
df.loc[:12,]

0
mean monthly: 2421.72


,amortization,interest+principal,interest,cumulative_interest,principal,cumulative_principal,hoa,cumulative_hoa,tax,cumulative_tax,...,expense,cumulative_expense,alternative,cumulative_alternative,balance,balance_check,expected_sale,down,min_sale_price,milestones
month,,,,,,,,,,,,,,,,,,,,,
1,2421.720982,2421.720982,1663.020833,1663.020833,758.700149,758.700149,321.0,321.0,586.666667,586.666667,...,-929.312500,-929.312500,3500.0,3500.0,513241.299851,510712.541282,700000.0,125000.0,680021.390194,
2,2421.720982,2421.720982,1657.341697,3320.362531,764.379284,1523.079433,321.0,642.0,586.666667,1173.333333,...,-934.991636,-1864.304136,3500.0,7000.0,511476.920567,508936.767048,700000.0,125000.0,677179.947120,
3,2421.720982,2421.720982,1651.644223,4972.006753,770.076759,2293.156192,321.0,963.0,586.666667,1760.000000,...,-940.689111,-2804.993247,3500.0,10500.0,509706.843808,507155.258543,700000.0,125000.0,674326.509363,
4,2421.720982,2421.720982,1645.928350,6617.935103,775.792632,3068.948824,321.0,1284.0,586.666667,2346.666667,...,-946.404984,-3751.398230,3500.0,14000.0,507931.051176,505367.997251,700000.0,125000.0,671461.038188,
5,2421.720982,2421.720982,1640.194019,8258.129123,781.526962,3850.475787,321.0,1605.0,586.666667,2933.333333,...,-952.139314,-4703.537544,3500.0,17500.0,506149.524213,503574.964593,700000.0,125000.0,668583.494739,
6,2421.720982,2421.720982,1634.441172,9892.570295,787.279810,4637.755597,321.0,1926.0,586.666667,3520.000000,...,-957.892161,-5661.429705,3500.0,21000.0,504362.244403,501776.141934,700000.0,125000.0,665693.840033,
7,2421.720982,2421.720982,1628.669748,11521.240042,793.051234,5430.806831,321.0,2247.0,586.666667,4106.666667,...,-963.663586,-6625.093291,3500.0,24500.0,502569.193169,499971.510577,700000.0,125000.0,662792.034959,
8,2421.720982,2421.720982,1622.879686,13144.119728,798.841296,6229.648127,321.0,2568.0,586.666667,4693.333333,...,-969.453647,-7594.546938,3500.0,28000.0,500770.351873,498161.051765,700000.0,125000.0,659878.040282,
9,2421.720982,2421.720982,1617.070928,14761.190656,804.650054,7034.298181,321.0,2889.0,586.666667,5280.000000,...,-975.262405,-8569.809344,3500.0,31500.0,498965.701819,496344.746680,700000.0,125000.0,656951.816641,


In [19]:
import plotly.graph_objects as go

In [111]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = df.index, y = df.equity, name = 'equity'))
fig.add_trace(go.Scatter(x = df.index, y = df.cumulative_expense, name = 'cumulative_expense'))
fig.add_trace(go.Scatter(x = df.index, y = df.cumulative_principal, name = 'cumulative_principal'))
fig.add_trace(go.Scatter(x = df.index, y = df.down, name = 'down')) # , fill='tozeroy'
fig.add_trace(go.Scatter(x = df.index, y = df.cumulative_tax, name = 'cumulative_tax'))
fig.add_trace(go.Scatter(x = df.index, y = df.cumulative_hoa, name = 'cumulative_hoa'))
fig.add_trace(go.Scatter(x = df.index, y = df.min_sale_price, name = 'min_sale_price'))
fig.add_trace(go.Scatter(x = df.index, y = df.expected_sale, name = 'expected_sale'))
fig.add_trace(go.Scatter(x = df.index, y = df.cumulative_alternative, name = 'cumulative_alternative'))
# fig.add_vrect(x0="24", x1="100", 
#               annotation_text="decline", annotation_position="top left",
#               annotation=dict(font_size=12, font_family="Verdana"),
#               fillcolor="green", opacity=0.1, line_width=0)


In [100]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = df.index, y = df.cumulative_principal, name = 'cumulative_principal'))
fig.add_trace(go.Scatter(x = df.index, y = df.cumulative_interest, name = 'cumulative_interest'))
fig.add_trace(go.Scatter(x = df.index, y = df.down, name = 'down'))
df['position'] = df.cumulative_principal - df.cumulative_interest + df.cumulative_deduction
fig.add_trace(go.Scatter(x = df.index, y = df.position, name = 'position'))
fig.show()

In [62]:

fig = go.Figure()
fig.add_trace(go.Scatter(x = df.index, y = df.principal, name = 'principal'))
fig.add_trace(go.Scatter(x = df.index, y = df.interest, name = 'interest'))
fig.add_trace(go.Scatter(x = df.index, y = df['interest+principal'], name = 'interest+principal'))
fig.show()